In [ ]:
# Import packages needed 
import win32com.client as win32


Application = win32.gencache.EnsureDispatch('Apwn.Document')
import numpy as np
import pandas as pd 

Application.InitFromFile(r"C:\Users\lumed\Desktop\Research\Aspen Python\ATP to Gasoline April 10_2020\ATP to Gasoline April 10_2020.bkp")

Application.Visible = True
Application.SuppressDialogs = 1
# Application.Reinit()
# Application.Run2()

FeedStockSet = pd.read_excel(r"C:\Users\lumed\Desktop\Research\Feedstocks\feedstock normalized.xlsx")

In [ ]:
# This code is a function that predicts the 42 products of the pyrolysis products based on the inoputs in aspen 
def pyrolysis(feedstock_comp):
    import pandas as pd
    import pickle
    
    
# The next line imports the ranzi data to extract the products 
    products = ['CELL1','HCE1','LIGC1','LIGH1','LIGO1','CELLA','HCEA1','HCEA2','LIG','LIGCC','LIGOH','GCO2','GCO','GCOH2','GH2','Char','HAA','GLYOX','C3H6O','C3H4O2','HMFU','LVG', 'XYL','pCOUMARYL','PHENOL','FE2MACR','CH3CHO','ETOH','CH3OH','CO','CO2','CH4','CH2O','H2','C2H4','H2O']
    # The next line creates an empty list, the products predicted are going to be stored in this list while we loop through each product model    
    predictions =[]

# The for loop will loop through the products models that was trained and saved earlier, predict and append each product to the prediction list 
    for name in products:
        model = 'model' + name + ".sav"
        model2 = pickle.load(open(model,'rb'))
        pred = model2.predict(feedstock_comp)
        predictions.append(pred)
    return (predictions)

In [ ]:
# This code adds gasoline and diesel columns to the data set

FeedStockSet['Diesel'] = np.zeros(len(FeedStockSet))
FeedStockSet['Gasoline'] = np.zeros(len(FeedStockSet))
# FeedStockSet.iat[0,6] = 2
FeedStockSet.head()

In [ ]:
# Create an empty list for diesel and gasoline yields 
Diesel_list = []
Gasoline_list = []

# This code will loop through all the feedstocks, predict the pyrolysis yields and update the diesel and gasoline yields for each feedstock 

for i in range(len(FeedStockSet)):
    feedstock = FeedStockSet.iloc[i,1:6].values
    for j in range(5):
        Application.Tree.FindNode("\Data\Streams\STOVER\Input\FLOW\MIXED").Elements.Item(j).Value = FeedStockSet.iloc[i,j+1]
    Pyrolysis_yield = pyrolysis(feedstock)
    print("Progress {:2.1%}".format(i / 341), end="\r")
#     print(i)
    
    for prod in range(len(Pyrolysis_yield)):
        if prod <= 10:
            Application.Tree.FindNode("\Data\Blocks\PYROLYSI\Data\Blocks\B1\Input\MOLE_YIELD").Elements.Item(0,prod).Value = round(Pyrolysis_yield[prod],3)
        else:
            Application.Tree.FindNode("\Data\Blocks\PYROLYSI\Data\Blocks\B1\Input\MOLE_YIELD").Elements.Item(0,prod).Value = round(Pyrolysis_yield[prod],7)
    Application.Run2()
    Gasoline = Application.Tree.FindNode("\Data\Streams\MOGAS\Output\MASSFLMX\$TOTAL").Value
    Diesel = Application.Tree.FindNode("\Data\Streams\DIESEL\Output\MASSFLMX\$TOTAL").Value
    
    FeedStockSet.iat[i,6] = Gasoline 
    FeedStockSet.iat[i,7] = Diesel
    
    Diesel_list.append(Diesel)
    Gasoline_list.append(Gasoline)
    Application.Reinit()
FeedStockSet['Diesel'] = Diesel_list
FeedStockSet['Gasoline'] = Gasoline_list

FeedStockSet.to_excel(r"C:\Users\lumed\Desktop\Research\Feedstocks\feedstock normalized3.xlsx" ,engine = 'xlsxwriter')

In [ ]:
FeedStockSet.head()